In [2]:
import pandas as pd

In [3]:
x = pd.read_csv('./features_reduced_imputed.csv')
x.drop(['Unnamed: 0'], axis = 1, inplace = True)
x.shape

(1700, 92)

# 1. binary column중 unbalance한 Feature 삭제 therehold = 0.01

In [5]:
binary_column_info = {}

cnt = 0

for column in x.columns:
    # 유일한 값들 확인
    unique_values = x[column].dropna().unique()
    
    # 0과 1만 있는지 확인
    if set(unique_values) <= {0, 1}:
        count_0 = (x[column] == 0).sum()
        count_1 = (x[column] == 1).sum()
        total_count = count_0 + count_1
        percent_0 = (count_0 / total_count) * 100
        percent_1 = (count_1 / total_count) * 100
        binary_column_info[column] = {
            '0 count': count_0, 
            '1 count': count_1,
            '0 percent': percent_0,
            '1 percent': percent_1
        }

li = []
for column, counts in binary_column_info.items():

    if(counts['1 percent'] <= 1):
        print(column, "is unbalanced.", counts['0 count'], counts['1 count'])
        cnt +=1
        li.append(column)
    else:
        print(column, "is balanced.", counts['0 count'], counts['1 count'])

print(cnt)

SEX is balanced. 635 1065
SIM_GIPERT is balanced. 1643 57
nr_11 is balanced. 1658 42
nr_01 is unbalanced. 1696 4
nr_02 is balanced. 1681 19
nr_03 is balanced. 1665 35
nr_04 is balanced. 1671 29
nr_07 is unbalanced. 1699 1
nr_08 is unbalanced. 1696 4
np_01 is unbalanced. 1698 2
np_04 is unbalanced. 1697 3
np_05 is unbalanced. 1689 11
np_07 is unbalanced. 1699 1
np_08 is unbalanced. 1694 6
np_09 is unbalanced. 1698 2
np_10 is unbalanced. 1697 3
endocr_01 is balanced. 1472 228
endocr_02 is balanced. 1658 42
endocr_03 is unbalanced. 1687 13
zab_leg_01 is balanced. 1564 136
zab_leg_02 is balanced. 1579 121
zab_leg_03 is balanced. 1663 37
zab_leg_04 is unbalanced. 1691 9
zab_leg_06 is balanced. 1678 22
O_L_POST is balanced. 1590 110
K_SH_POST is balanced. 1654 46
SVT_POST is unbalanced. 1692 8
GT_POST is unbalanced. 1692 8
FIB_G_POST is unbalanced. 1685 15
IM_PG_P is balanced. 1650 50
ritm_ecg_p_04 is balanced. 1677 23
ritm_ecg_p_06 is unbalanced. 1699 1
ritm_ecg_p_08 is balanced. 1654 46
n_

In [6]:
x = x.drop(columns=li)
x.shape

(1700, 61)

In [7]:
x.to_csv('./1_1.csv')

# 2. VAE Test

In [37]:
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv('./1_1.csv')
data = data.drop(columns=['Unnamed: 0'])

# Normalize the data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# Convert normalized data back to a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=data.columns)

In [61]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the data
file_path = './1_1.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the data
print(data.head())

# Preprocess the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.drop(columns=['Unnamed: 0']))

# Split the data into training and validation sets
x_train, x_val = train_test_split(data_scaled, test_size=0.2, random_state=42)

# Define the encoder
input_dim = x_train.shape[1]
latent_dim = 2  # You can adjust the dimensionality of the latent space

inputs = layers.Input(shape=(input_dim,))
h = layers.Dense(64, activation='relu')(inputs)
h = layers.Dense(32, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_var = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder
decoder_input = layers.Input(shape=(latent_dim,))
decoder_h = layers.Dense(32, activation='relu')
decoder_h2 = layers.Dense(64, activation='relu')
decoder_mean = layers.Dense(input_dim, activation='sigmoid')

_h_decoded = decoder_h(decoder_input)
_h_decoded = decoder_h2(_h_decoded)
_outputs = decoder_mean(_h_decoded)

decoder = models.Model(decoder_input, _outputs)

# Define the VAE model
h_decoded = decoder_h(z)
h_decoded = decoder_h2(h_decoded)
outputs = decoder_mean(h_decoded)
vae = models.Model(inputs, outputs)

# Define the loss
reconstruction_loss = tf.keras.losses.mse(inputs, outputs)
reconstruction_loss *= input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE model
vae.fit(x_train, epochs=50, batch_size=32, validation_data=(x_val, None))

# Generate new samples using the trained VAE
def generate_samples(decoder, num_samples, latent_dim):
    z_sample = np.random.normal(size=(num_samples, latent_dim))
    generated_data = decoder.predict(z_sample)
    return scaler.inverse_transform(generated_data)

# Generate 100 new samples
new_samples = generate_samples(decoder, 10, latent_dim)

# Convert generated samples to DataFrame and round all values to integers
new_samples_df = pd.DataFrame(new_samples, columns=data.columns[1:])
#new_samples_df = new_samples_df.round(1)

# Display the generated samples
print(new_samples_df.head())


   Unnamed: 0   AGE  SEX  INF_ANAM  IBS_POST  SIM_GIPERT  ZSN_A  nr_11  nr_02  \
0           0  77.0    1         2         2           0      0      0      0   
1           1  55.0    1         1         0           0      0      0      0   
2           2  52.0    1         0         2           0      0      0      0   
3           3  68.0    0         0         2           0      1      0      0   
4           4  60.0    1         0         2           0      0      0      0   

   nr_03  ...  NOT_NA_1_n  NOT_NA_2_n  NOT_NA_3_n  LID_S_n  B_BLOK_S_n  \
0      0  ...           0           0           0        1           0   
1      0  ...           1           0           0        1           0   
2      0  ...           3           2           2        1           1   
3      0  ...           0           0           0        0           0   
4      0  ...           0           0           0        0           0   

   ANT_CA_S_n  GEPAR_S_n  ASP_S_n  TIKL_S_n  TRENT_S_n  
0          

In [62]:
#값 분포 확인
for column in new_samples_df.columns:
    unique_values = new_samples_df[column].dropna().unique()
    print(column, unique_values)

AGE [61.862152 61.904064 61.873535 61.8622   61.88735  61.86219  62.29251
 61.872353 61.862274 61.862232]
SEX [0.62647057 0.6264798  0.6264786  0.6264807  0.6273904  0.6264753 ]
INF_ANAM [0.5554382  0.56035745 0.56237835 0.5555472  0.5624952  0.555519
 0.60098267 0.5607117  0.5557131  0.5556157 ]
IBS_POST [1.1676898 1.177472  1.1714579 1.1677295 1.1740502 1.1677223 1.2279601
 1.1714482 1.1677824 1.1677508]
SIM_GIPERT [0.03352957 0.0335954  0.03359849 0.0335298  0.0336033  0.03352977
 0.03554852 0.03359427 0.03353011 0.03352992]
ZSN_A [0.19178294 0.19483143 0.19401135 0.19180085 0.19538674 0.19179365
 0.21972127 0.19313085 0.19183792 0.19181687]
nr_11 [0.02470591 0.02473683 0.02472922 0.02470594 0.0247322  0.02470592
 0.02584688 0.02472484 0.02470601 0.02470597]
nr_02 [0.01117648 0.01119135 0.01118796 0.01117649 0.01119956 0.01117649
 0.01187023 0.01118534 0.01117652 0.0111765 ]
nr_03 [0.02058899 0.02081238 0.02075322 0.02058991 0.02079522 0.02058982
 0.0244072  0.02075441 0.02059102 0.

In [17]:
combined_df = pd.concat([data, new_samples_df], ignore_index=True)
combined_df = combined_df.drop(columns=['Unnamed: 0'])
combined_df.to_csv('1_2.csv')